# Indexing and Selecting data


---

## Learning Objectives 


- Select data by position using `.isel()` with values or slices
- Select data by coordinate label/value using `.sel()` with values or slices
- Use nearest-neighbor lookups with `.sel()`
- Use `interp()` to interpolate by coordinate labels

## Prerequisites


| Concepts | Importance | Notes |
| --- | --- | --- |
| [Understanding of xarray core data structures](./01-xarray-fundamentals.ipynb) | Necessary | |
| [Basic familiarity with NumPy indexing](https://numpy.org/doc/stable/reference/arrays.indexing.html) | Helpful | |
| [Basic familiarity with Pandas indexing](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html) | Helpful | |

- **Time to learn**: *medium*



---

## Imports


In [ ]:
import xarray as xr

In [ ]:
ds = xr.open_dataset(
    "./data/tas_Amon_CESM2_historical_r11i1p1f1_gn_200001-201412.nc", engine="netcdf4"
)
ds

## NumPy Positional Indexing

When working with numpy, indexing is done by position (slices/ranges/scalars).

In [ ]:
temp = ds["tas"].data  # retrieve numpy array
temp

In [ ]:
temp.shape, temp.ndim

Let's extract a timeseries for a single spatical location 


In [ ]:
temp[:, 20, 40]

<div class="admonition alert alert-warning">
    <p class="admonition-title" style="font-weight:bold"></p>
    but wait, what labels go with 20 and 40? Was that lat/lon or lon/lat? Where are the timestamps that go along with this time-series?
</div>

## Different choices for indexing 


Xarray supports two kinds of indexing 

- Positional indexing via `.isel()`: provides primarily integer position based index (from `0` to `length-1` of the axis/dimension
- Label indexing via `.sel()`: provides primarily label based index

Xarray's indexing methods preserves the coordinate labels and associated metadata



### Selection by position

The `.isel()` method is the primary access method for **purely integer based indexing**. The following are valid inputs:
- An integer e.g. `lat=10`
- A list or array of integers `lon=[10, 20, 39]`
- A slice object with integers e.g. `time=slice(2, 20)`

In [ ]:
ds.tas.isel()  # is the data as the original object i.e. no selection

In [ ]:
ds.tas.isel(lat=100)  # Equivalent to ds.tas.isel(**{'lat': 100})

In [ ]:
ds.tas.isel(**{'lat': 100})

In [ ]:
ds.tas.isel(lat=100, time=[-2, -1])

In [ ]:
ds.tas.isel(lon=100, time=slice(10, 20))

### Selection by label 


The `.sel()` method is the primary access method for **purely coordinate label based indexing.**. The following are valid inputs:

- A single coordinate label e.g. `time="2021-03-01"`
- A list or array of coordinate labels `lon=[="2021-01-01", ="2021-03-10", ="2021-03-12"]`
- A slice object with coordinate labels e.g. `time=slice("2021-01-01", "2021-03-01")`.  (Note that contrary to usual Python slices, both the start and the stop are included, when present in the index!)

In [ ]:
ds.tas.sel(time="2013")

It is also possible to use slice for the time dimension:

In [ ]:
ds.tas.sel(time=slice("2013-01-01", "2014-12-31"))


<div class="admonition alert alert-warning">
    <p class="admonition-title" style="font-weight:bold">Warning: Be careful when working with floating coordinate labels</p>
        
    When we have integer or string values for coordinate labels, "sel()" works flawlessly. When we try to work with floating coordinate labels, things get a little tricky:
        
</div>




In [ ]:
ds.tas.sel(lat=39.5, lon=105.7)

### Nearest-neighbor lookups

As shown above, when our coordinate labels are not integers or strings but floating point numbers, `.sel()` may throw a `KeyError`:

`ds.tas.sel(lat=39.5, lon=105.7)` fails because we are trying to use a conditional for an approximate value i.e floating numbers are represented approximately inside the computer, and xarray is unable to locate this exact value. To address this issue, xarray supports `method` and `tolerance` keyword argument. The `method` parameter allows for enabling nearest neighbor (inexact) lookups by use of the methods `'pad', 'backfill' or 'nearest'`: 

In [ ]:
ds.tas.sel(lat=39.5, lon=105.7, method='nearest')

So the closest location in the data was at `lat=39.11`, `lon=106.2`.

**Tip:** Another way to use the nearest neighbor lookup is via slice objects. For e.g.:

In [ ]:
ds.tas.sel(lat=slice(39, 39.5))

Operators can be chained, so multiple operations can be peformed sequentially. For example, to select an area of interest and the first time index

In [ ]:
ds.tas.isel(time=0).sel(lon=slice(20, 160), lat=slice(-80, 25))

### Interpolation

If we want to interpolate along coordinates rather than looking up the nearest neighbos, we can use the `.interp()` method. To use `interp()` requires the presence of `scipy` library. 


In [ ]:
ds.tas.interp(lat=[10, 10.1, 10.2], method='nearest')

---

In [ ]:
%load_ext watermark
%watermark --time --python --updated --iversion

## Resources and References

- [Xarray Documentation - Indexing and Selecting Data](https://xarray.pydata.org/en/stable/indexing.html)
- [Xarray Documentation - Interpolation](https://xarray.pydata.org/en/stable/user-guide/interpolation.html)


<div class="admonition alert alert-success">
    <p class="title" style="font-weight:bold">Previous: <a href="./01-xarray-fundamentals.ipynb">Xarray Fundamentals</a></p>
         <p class="title" style="font-weight:bold">Next: <a href="./03-data-visualization.ipynb">Data Visualization</a></p>
</div>